# Baseline Model using Graph Stat for binary

## Set up

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import json
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score


In [3]:
PATH = Path.cwd().parents[2]
DATA_PATH = os.path.join(PATH, 'data/processed/ir_gigahorse')

## List Traditional ML model to compare

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

models = {
  "LogisticRegression()": LogisticRegression(),
  "DecisionTreeClassifier()": DecisionTreeClassifier(),
  "RandomForestClassifier()": RandomForestClassifier(),
  "AdaBoostClassifier()": AdaBoostClassifier(),
  "ExtraTreesClassifier()": ExtraTreesClassifier(),
  "XGBClassifier()": XGBClassifier(),
  "LGBMClassifier()": LGBMClassifier(),
  "SVC()": SVC(),
  "GaussianNB()": GaussianNB(),
  "KNeighborsClassifier()": KNeighborsClassifier(),
  "SGDClassifier()": SGDClassifier(),
  "MLPClassifier()": MLPClassifier(),
}

## Load Dataset

In [5]:
df = pd.read_csv(os.path.join(DATA_PATH, 'graph_stat_dataset.csv')).set_index('address')

with open(os.path.join(DATA_PATH, 'graph_stat_features.json'), "r") as f:
    features = json.load(f)

with open(os.path.join(DATA_PATH, 'labels.json'), "r") as f:
    labels = json.load(f)

In [6]:
df.head()

,mint,leak,limit,num_nodes,num_edges,avg_degree,density,connected_components,avg_clustering
address,,,,,,,,,
0x2753dce37a7edb052a77832039bcc9aa49ad8b25,0,0,1,3529,3528,1.999433,0.000283,1,0.0
0x94b7d24552933f50a5a5705c446528806dcea381,0,0,0,67,66,1.970149,0.014925,1,0.0
0xe0b9d4146aad6936cbfcbe4dae47e34aab96b093,0,0,0,4091,4090,1.999511,0.000244,1,0.0
0x10f6f2b97f3ab29583d9d38babf2994df7220c21,1,0,1,6037,6036,1.999669,0.000166,1,0.0
0x11cbc781dadaad13fc3a361772c80b1c027820af,0,0,0,4827,4826,1.999586,0.000207,1,0.0


In [7]:
features[:5]

['num_nodes', 'num_edges', 'avg_degree', 'density', 'connected_components']

In [8]:
labels[:5]

['mint', 'leak', 'limit']

## Prepare Features and Labels

In [9]:
X = df[features]
y_mint = df[['mint']]
y_leak = df[['leak']]
y_limit = df[['limit']]

## Train/Test Split

In [10]:
X_mint_train, X_mint_test, y_mint_train, y_mint_test = train_test_split(
    X, y_mint, test_size=0.2, random_state=42
)

X_leak_train, X_leak_test, y_leak_train, y_leak_test = train_test_split(
    X, y_leak, test_size=0.2, random_state=42
)

X_limit_train, X_limit_test, y_limit_train, y_limit_test = train_test_split(
    X, y_limit, test_size=0.2, random_state=42
)

## Run all models and collect reports

In [11]:
def get_report_all_ml(X_train, y_train, X_test, y_test):
    report_list = []

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Generate classification report (as dict)
        report_dict = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

        # Average scores across all labels (macro average)
        avg_scores = report_dict["macro avg"]

        report_list.append({
            "Model": name,
            "Precision": avg_scores["precision"],
            "Recall": avg_scores["recall"],
            "F1-score": avg_scores["f1-score"]
        })

    df_report = pd.DataFrame(report_list)
    df_report = df_report.sort_values("F1-score", ascending=False).reset_index(drop=True)

    return df_report

In [12]:
def display_cm(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0, 1])
    return cm_display

## Show report

In [13]:
data = {
  'mint': {
    "X_train": X_mint_train,
    "X_test": X_mint_test,
    "y_train": y_mint_train,
    "y_test": y_mint_test
  },
  'leak': {
    "X_train": X_leak_train,
    "X_test": X_leak_test,
    "y_train": y_leak_train,
    "y_test": y_leak_test
  },
  'limit': {
    "X_train": X_limit_train,
    "X_test": X_limit_test,
    "y_train": y_limit_train,
    "y_test": y_limit_test
  }
}

reports = []

for d in data.values():
    X_train = d['X_train']
    X_test = d['X_test']
    y_train = d['y_train']
    y_test = d['y_test']
    df_report = get_report_all_ml(X_train, y_train, X_test, y_test)
    reports.append(df_report)

[LightGBM] [Info] Number of positive: 17, number of negative: 36
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000065 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76
[LightGBM] [Info] Number of data points in the train set: 53, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.320755 -> initscore=-0.750306
[LightGBM] [Info] Start training from score -0.750306
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [14]:
reports[0]

,Model,Precision,Recall,F1-score
0,GaussianNB(),0.642857,0.791667,0.590643
1,XGBClassifier(),0.642857,0.791667,0.590643
2,SVC(),0.625000,0.750000,0.533333
3,DecisionTreeClassifier(),0.611111,0.708333,0.475936
4,AdaBoostClassifier(),0.611111,0.708333,0.475936
5,KNeighborsClassifier(),0.611111,0.708333,0.475936
6,MLPClassifier(),0.428571,0.500000,0.461538
7,RandomForestClassifier(),0.600000,0.666667,0.416667
8,ExtraTreesClassifier(),0.600000,0.666667,0.416667
9,LogisticRegression(),0.409091,0.375000,0.391304


In [15]:
reports[1]

,Model,Precision,Recall,F1-score
0,GaussianNB(),0.742424,0.700,0.714286
1,LogisticRegression(),0.625000,0.575,0.575758
2,XGBClassifier(),0.625000,0.575,0.575758
3,SVC(),0.357143,0.500,0.416667
4,MLPClassifier(),0.357143,0.500,0.416667
5,SGDClassifier(),0.357143,0.500,0.416667
6,LGBMClassifier(),0.357143,0.500,0.416667
7,DecisionTreeClassifier(),0.346154,0.450,0.391304
8,AdaBoostClassifier(),0.346154,0.450,0.391304
9,RandomForestClassifier(),0.346154,0.450,0.391304


In [16]:
reports[2]

,Model,Precision,Recall,F1-score
0,DecisionTreeClassifier(),0.714286,0.714286,0.714286
1,AdaBoostClassifier(),0.714286,0.714286,0.714286
2,ExtraTreesClassifier(),0.714286,0.714286,0.714286
3,RandomForestClassifier(),0.645833,0.642857,0.641026
4,KNeighborsClassifier(),0.500000,0.500000,0.497436
5,XGBClassifier(),0.500000,0.500000,0.497436
6,GaussianNB(),0.422222,0.428571,0.416667
7,LGBMClassifier(),0.422222,0.428571,0.416667
8,SGDClassifier(),0.250000,0.500000,0.333333
9,SVC(),0.250000,0.500000,0.333333


## MLP

### Build model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import MeanSquaredError

2025-07-15 15:00:14.418433: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
def Model(input_dim, output_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(512, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.4),

        Dense(256, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.3),

        Dense(128, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.2),

        Dense(output_dim, activation='sigmoid')  # sigmoid for multi-label
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=1e-6),
        metrics=['accuracy']
    )
    return model

In [19]:
model = Model(input_dim=X.shape[1], output_dim=1)

### Train

In [20]:
model.fit(X_mint_train, y_mint_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 300ms/step - accuracy: 0.6473 - loss: 0.5643 - val_accuracy: 0.7273 - val_loss: 3.0794 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.7262 - loss: 0.5655 - val_accuracy: 0.7273 - val_loss: 2.1554 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.6577 - loss: 0.5970 - val_accuracy: 0.7273 - val_loss: 1.6730 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5739 - loss: 0.6816 - val_accuracy: 0.7273 - val_loss: 1.4585 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5630 - loss: 0.6842 - val_accuracy: 0.7273 - val_loss: 1.3140 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7842 - loss: 0.6357 - val_accuracy: 0.7273 - val_loss: 1.2078 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.6265 - los

### Predict

In [21]:
y_mint_test_prob = model.predict(X_mint_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step


In [22]:
def tune_thresholds(y_true, y_pred_prob, metric='f1'):
    y_true = np.asarray(y_true)          # Fix: convert to NumPy
    y_pred_prob = np.asarray(y_pred_prob)

    best_thresholds = []
    best_scores = []

    for i in range(y_true.shape[1]):
        label_true = y_true[:, i]
        label_probs = y_pred_prob[:, i]  # Fix here too

        thresholds = np.linspace(0.0, 1.0, 101)
        scores = []

        for t in thresholds:
            label_pred = (label_probs >= t).astype(int)
            if metric == 'f1':
                score = f1_score(label_true, label_pred, zero_division=0)
            scores.append(score)

        best_t = thresholds[np.argmax(scores)]
        best_score = np.max(scores)

        best_thresholds.append(best_t)
        best_scores.append(best_score)

        print(f"Label {i}: Best threshold = {best_t:.2f}, Best {metric} = {best_score:.4f}")

    return best_thresholds, best_scores


In [23]:
best_thresholds, _ = tune_thresholds(y_mint_test, y_mint_test_prob)

Label 0: Best threshold = 0.49, Best f1 = 0.6667


In [24]:
y_mint_pred = (model.predict(X_mint_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


### Report

In [25]:
print(classification_report(y_mint_test, y_mint_pred))

              precision    recall  f1-score   support

           0       1.00      0.83      0.91        12
           1       0.50      1.00      0.67         2

    accuracy                           0.86        14
   macro avg       0.75      0.92      0.79        14
weighted avg       0.93      0.86      0.87        14



## Autoencoder + MPL

In [26]:
def Autoencoder(input_dim=256):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128),
        LeakyReLU(0.01),

        Dense(32),
        BatchNormalization(),
        LeakyReLU(0.01),

        Dense(128),
        BatchNormalization(),
        LeakyReLU(0.01),

        Dense(input_dim, activation='sigmoid')  # sigmoid for multi-label
    ])

    model.compile(optimizer=Adam(1e-6), loss=MeanSquaredError())
    return model


In [27]:
autoencoder = Autoencoder(input_dim=X.shape[1])
autoencoder.fit(X_mint_train, X_mint_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - loss: 7008409.5000 - val_loss: 4984416.0000
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 6993393.0000 - val_loss: 4984378.0000
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 7078381.5000 - val_loss: 4984350.5000
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 6631401.0000 - val_loss: 4984314.0000
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 6576142.5000 - val_loss: 4984284.5000
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 6741582.0000 - val_loss: 4984256.5000
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 6972119.0000 - val_loss: 4984230.0000
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 6624101.5000 - val_loss: 4984203.5000
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 7060374.0000 - val_loss: 4984182.0000
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 7397174.5000 - val_loss: 4984166.0000
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 7

In [28]:
X_train_encoded = autoencoder.predict(X_mint_train)
X_test_encoded = autoencoder.predict(X_mint_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [29]:
model = Model(input_dim=X_train_encoded.shape[1], output_dim=1)

In [30]:
model.fit(X_train_encoded, y_mint_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 287ms/step - accuracy: 0.3318 - loss: 1.1170 - val_accuracy: 0.2727 - val_loss: 0.7247 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3368 - loss: 1.3286 - val_accuracy: 0.2727 - val_loss: 0.7177 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.3527 - loss: 1.0901 - val_accuracy: 0.2727 - val_loss: 0.7122 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3631 - loss: 1.2044 - val_accuracy: 0.3636 - val_loss: 0.7084 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3160 - loss: 1.2031 - val_accuracy: 0.3636 - val_loss: 0.7061 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3844 - loss: 1.1326 - val_accuracy: 0.2727 - val_loss: 0.7060 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2842 - loss

In [31]:
y_pred_prob = model.predict(X_test_encoded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step


In [32]:
best_thresholds, _ = tune_thresholds(y_mint_test, y_pred_prob)

Label 0: Best threshold = 0.50, Best f1 = 0.2857


In [33]:
y_pred = (model.predict(X_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


### Report

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.50      1.00      0.67         7

    accuracy                           0.50        14
   macro avg       0.25      0.50      0.33        14
weighted avg       0.25      0.50      0.33        14

